In [1]:
%cd ~/gymnasium-test/
# %load_ext line_profiler
# %load_ext memory_profiler
# %load_ext autoreload
# %autoreload 2

/home/n.saumik/gymnasium-test


In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["MPLCONFIGDIR"] = "/tmp/"
import tensorflow as tf

tf.config.set_visible_devices([], "GPU")
tf.get_logger().setLevel("ERROR")

import numpy as np
from tqdm import tqdm

from tree import Tree, make_model, Forest, EarlyStopper
np.set_printoptions(linewidth=200)

import pandas as pd

importing numpy
importing tensorflow
done with imports


2024-11-09 02:06:57.089312: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
threshes = [0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 0.99, 1]

def get_diff(x, steps, nodes=32, layers=2, seeds=10, verbose=False):
    
    x = int(x/4)
    
    all_pcts = []
    
    for seed in tqdm(range(seeds), desc=f"{x},{steps},{nodes},{layers}", disable=verbose):
    
        low = Forest(4, "crit-9-10-0-0.2", x*seed, x*(seed+1))
        low_X, low_Y = low.get_training_data("A", window=0)
        low_Y[:] = 0

        high = Forest(4, "crit-9-10-0-0.2", x*(seed+1), x*(seed+2))
        high_X, high_Y = high.get_training_data("D", window=0)
        high_Y[:] = 1

        train_X = tf.convert_to_tensor(np.concatenate([low_X, high_X]))
        train_Y = tf.convert_to_tensor(np.concatenate([low_Y, high_Y]))

        model = make_model(4, 0, num_nodes=nodes, num_layers=layers)

        ds = tf.data.Dataset.from_tensor_slices((train_X, train_Y))
        ds = ds.repeat().shuffle(100000).batch(32).take(steps)

        acc = model.fit(ds, verbose=verbose).history["accuracy"][0]
        low_pred = model.predict(low_X)
        pcts = [(low_pred[:,1] <= t).mean().round(3) for t in threshes]
        
        all_pcts.append(pcts)
    pcts = np.array(all_pcts)
    return pcts

    

In [8]:
results = {}

In [9]:
%%time
for data_size in [2**6, 2**8, 2**10, 2**12, 2**14, 2**16]:
    for nodes, layers in [(16,1), (32,2), (64,3), (128,4)]:
        for steps in [2**4, 2**8, 2**12]:
            pcts = get_diff(data_size, steps, nodes, layers, seeds=20, verbose=False)
            results[data_size, steps, nodes, layers] = pcts.mean(axis=0)
        for steps in [2**16]:
            pcts = get_diff(data_size, steps, nodes, layers, seeds=2, verbose=True)
            results[data_size, steps, nodes, layers] = pcts.mean(axis=0)
        print("*"*50)
    print("$"*50)
    print("*"*50)

16,4096,16,1: 100%|██████████| 20/20 [01:03<00:00,  3.17s/it]


65536/65536 [==============================] - 45s 674us/step - loss: 0.0176 - accuracy: 0.9953
**************************************************


16,4096,32,2: 100%|██████████| 20/20 [01:07<00:00,  3.37s/it]


65536/65536 [==============================] - 47s 704us/step - loss: 0.0100 - accuracy: 0.9962
**************************************************


16,4096,64,3: 100%|██████████| 20/20 [01:14<00:00,  3.70s/it]


...

65536/65536 [==============================] - 47s 718us/step - loss: 0.6434 - accuracy: 0.6106
**************************************************


16384,4096,64,3: 100%|██████████| 20/20 [02:58<00:00,  8.91s/it]


65536/65536 [==============================] - 53s 805us/step - loss: 0.6281 - accuracy: 0.6222
**************************************************


16384,4096,128,4: 100%|██████████| 20/20 [03:25<00:00, 10.28s/it]


65536/65536 [==============================] - 71s 1ms/step - loss: 0.6162 - accuracy: 0.6316
**************************************************
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
**************************************************
CPU times: user 2h 20min 44s, sys: 16min 23s, total: 2h 37min 8s
Wall time: 1h 54min 44s


In [10]:
df = []
for k,v in results.items():
    # print(f"{k[0]};{k[1]};{k[2]};{k[3]};{';'.join(v.round(3).astype(str))}")
    df.append(list(k) + list(v))
df = pd.DataFrame(df, columns=["x", "steps", "nodes", "layers"] + threshes)

In [21]:
# df.groupby(['steps', 'x']).apply(display)
df.round(3).groupby(['nodes', 'layers', 'steps']).apply(display)

,x,steps,nodes,layers,0,0.01,0.02,0.05,0.1,0.2,0.5,0.8,0.9,0.95,0.99,1
0,64,16,16,1,0.0,0.061,0.074,0.110,0.154,0.230,0.608,0.936,0.967,0.980,0.989,1.0
16,256,16,16,1,0.0,0.034,0.045,0.060,0.099,0.172,0.536,0.879,0.930,0.955,0.981,1.0
32,1024,16,16,1,0.0,0.038,0.053,0.079,0.105,0.140,0.482,0.841,0.887,0.927,0.971,1.0
48,4096,16,16,1,0.0,0.030,0.046,0.072,0.105,0.160,0.587,0.874,0.918,0.943,0.976,1.0
64,16384,16,16,1,0.0,0.029,0.040,0.069,0.104,0.143,0.484,0.851,0.894,0.924,0.969,1.0
80,65536,16,16,1,0.0,0.037,0.048,0.073,0.099,0.144,0.477,0.833,0.882,0.912,0.958,1.0


,x,steps,nodes,layers,0,0.01,0.02,0.05,0.1,0.2,0.5,0.8,0.9,0.95,0.99,1
1,64,256,16,1,0.0,0.131,0.201,0.322,0.364,0.399,0.806,0.999,1.000,1.000,1.000,1.0
17,256,256,16,1,0.0,0.022,0.036,0.062,0.096,0.172,0.658,0.995,1.000,1.000,1.000,1.0
33,1024,256,16,1,0.0,0.001,0.003,0.010,0.021,0.057,0.577,0.985,0.995,0.998,0.999,1.0
49,4096,256,16,1,0.0,0.001,0.002,0.005,0.011,0.027,0.580,0.983,0.994,0.998,1.000,1.0
65,16384,256,16,1,0.0,0.001,0.001,0.005,0.013,0.031,0.735,0.987,0.995,0.998,1.000,1.0
81,65536,256,16,1,0.0,0.000,0.001,0.004,0.010,0.027,0.695,0.986,0.995,0.998,1.000,1.0


,x,steps,nodes,layers,0,0.01,0.02,0.05,0.1,0.2,0.5,0.8,0.9,0.95,0.99,1
2,64,4096,16,1,0.0,0.444,0.490,0.576,0.698,0.867,0.993,1.000,1.0,1.0,1.0,1.0
18,256,4096,16,1,0.0,0.162,0.189,0.227,0.257,0.353,0.817,0.995,1.0,1.0,1.0,1.0
34,1024,4096,16,1,0.0,0.012,0.021,0.044,0.070,0.114,0.633,0.995,1.0,1.0,1.0,1.0
50,4096,4096,16,1,0.0,0.000,0.000,0.001,0.003,0.017,0.621,0.998,1.0,1.0,1.0,1.0
66,16384,4096,16,1,0.0,0.000,0.000,0.000,0.000,0.002,0.740,1.000,1.0,1.0,1.0,1.0
82,65536,4096,16,1,0.0,0.000,0.000,0.000,0.000,0.000,0.953,1.000,1.0,1.0,1.0,1.0


,x,steps,nodes,layers,0,0.01,0.02,0.05,0.1,0.2,0.5,0.8,0.9,0.95,0.99,1
3,64,65536,16,1,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.0
19,256,65536,16,1,0.042,0.734,0.844,0.956,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.0
35,1024,65536,16,1,0.000,0.147,0.186,0.262,0.346,0.485,0.809,0.954,0.979,0.992,0.998,1.0
51,4096,65536,16,1,0.000,0.017,0.024,0.038,0.069,0.141,0.602,0.987,1.000,1.000,1.000,1.0
67,16384,65536,16,1,0.000,0.000,0.000,0.002,0.027,0.088,0.614,0.998,1.000,1.000,1.000,1.0
83,65536,65536,16,1,0.000,0.000,0.000,0.002,0.016,0.088,0.841,1.000,1.000,1.000,1.000,1.0


,x,steps,nodes,layers,0,0.01,0.02,0.05,0.1,0.2,0.5,0.8,0.9,0.95,0.99,1
4,64,16,32,2,0.0,0.007,0.017,0.044,0.100,0.200,0.616,0.982,0.997,1.000,1.000,1.0
20,256,16,32,2,0.0,0.003,0.006,0.019,0.045,0.098,0.512,0.938,0.977,0.993,1.000,1.0
36,1024,16,32,2,0.0,0.004,0.009,0.022,0.043,0.091,0.496,0.945,0.980,0.989,0.999,1.0
52,4096,16,32,2,0.0,0.004,0.007,0.019,0.037,0.086,0.553,0.919,0.968,0.986,0.998,1.0
68,16384,16,32,2,0.0,0.002,0.005,0.012,0.029,0.070,0.501,0.933,0.973,0.988,0.998,1.0
84,65536,16,32,2,0.0,0.002,0.005,0.016,0.035,0.078,0.472,0.935,0.973,0.987,0.998,1.0


...

,x,steps,nodes,layers,0,0.01,0.02,0.05,0.1,0.2,0.5,0.8,0.9,0.95,0.99,1
13,64,256,128,4,0.0,0.123,0.239,0.368,0.396,0.400,0.874,1.000,1.0,1.0,1.0,1.0
29,256,256,128,4,0.0,0.000,0.003,0.051,0.123,0.220,0.710,0.997,1.0,1.0,1.0,1.0
45,1024,256,128,4,0.0,0.000,0.000,0.000,0.002,0.028,0.586,0.999,1.0,1.0,1.0,1.0
61,4096,256,128,4,0.0,0.000,0.000,0.000,0.000,0.002,0.600,1.000,1.0,1.0,1.0,1.0
77,16384,256,128,4,0.0,0.000,0.000,0.000,0.000,0.001,0.790,1.000,1.0,1.0,1.0,1.0
93,65536,256,128,4,0.0,0.000,0.000,0.000,0.000,0.001,0.852,1.000,1.0,1.0,1.0,1.0


,x,steps,nodes,layers,0,0.01,0.02,0.05,0.1,0.2,0.5,0.8,0.9,0.95,0.99,1
14,64,4096,128,4,0.0,0.986,0.998,1.000,1.000,1.000,1.000,1.000,1.000,1.0,1.0,1.0
30,256,4096,128,4,0.0,0.565,0.677,0.841,0.931,0.977,0.994,0.998,0.999,1.0,1.0,1.0
46,1024,4096,128,4,0.0,0.076,0.095,0.119,0.152,0.230,0.716,0.980,0.996,1.0,1.0,1.0
62,4096,4096,128,4,0.0,0.001,0.004,0.017,0.033,0.061,0.641,0.998,1.000,1.0,1.0,1.0
78,16384,4096,128,4,0.0,0.000,0.000,0.000,0.000,0.008,0.786,1.000,1.000,1.0,1.0,1.0
94,65536,4096,128,4,0.0,0.000,0.000,0.000,0.000,0.001,0.974,1.000,1.000,1.0,1.0,1.0


,x,steps,nodes,layers,0,0.01,0.02,0.05,0.1,0.2,0.5,0.8,0.9,0.95,0.99,1
15,64,65536,128,4,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.0,1.0
31,256,65536,128,4,0.016,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.0,1.0
47,1024,65536,128,4,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.0,1.0
63,4096,65536,128,4,0.000,0.636,0.694,0.766,0.817,0.866,0.948,0.988,0.996,0.998,1.0,1.0
79,16384,65536,128,4,0.000,0.086,0.116,0.152,0.194,0.275,0.707,0.978,0.996,0.999,1.0,1.0
95,65536,65536,128,4,0.000,0.078,0.104,0.134,0.158,0.213,0.810,0.992,0.999,1.000,1.0,1.0


/tmp/873550.tmpdir/ipykernel_2323336/450540414.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.round(3).groupby(['nodes', 'layers', 'steps']).apply(display)


""


In [19]:
df[df.nodes == 64].pivot(index='x', columns='steps', values=0.2)

steps,16,256,4096,65536
x,,,,
64,0.11140,0.40715,1.00000,1.0000
256,0.04140,0.21930,0.78590,1.0000
1024,0.01735,0.03690,0.19115,1.0000
4096,0.02410,0.00730,0.05550,0.4880
16384,0.02225,0.00485,0.00610,0.2040
65536,0.02435,0.00615,0.00170,0.1825


In [20]:
df[df.nodes == 128].pivot(index='x', columns='steps', values=0.2)

steps,16,256,4096,65536
x,,,,
64,0.03540,0.39985,1.00000,1.0000
256,0.00735,0.22045,0.97670,1.0000
1024,0.00225,0.02780,0.23015,1.0000
4096,0.00220,0.00250,0.06130,0.8665
16384,0.00310,0.00085,0.00780,0.2750
65536,0.00250,0.00065,0.00130,0.2130


In [5]:
%%time
get_diff(64, 2**4)
get_diff(64, 2**8)
get_diff(64, 2**12)
get_diff(64, 2**16)
get_diff(64, 2**20)

16/16 [==============================] - 0s 840us/step - loss: 0.8353 - accuracy: 0.4980
Steps 16: 0.498


{0: 0.0,
 0.01: 0.0,
 0.02: 0.0,
 0.05: 0.0,
 0.1: 0.031,
 0.2: 0.047,
 0.5: 0.745,
 0.8: 0.974,
 0.9: 0.995,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


256/256 [==============================] - 0s 691us/step - loss: 0.6011 - accuracy: 0.6309
Steps 256: 0.631


{0: 0.0,
 0.01: 0.026,
 0.02: 0.047,
 0.05: 0.047,
 0.1: 0.151,
 0.2: 0.234,
 0.5: 0.458,
 0.8: 1.0,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


4096/4096 [==============================] - 3s 726us/step - loss: 0.4052 - accuracy: 0.8138
Steps 4096: 0.814


...


65536/65536 [==============================] - 46s 700us/step - loss: 0.0383 - accuracy: 0.9858
Steps 65536: 0.986


{0: 0.016,
 0.01: 1.0,
 0.02: 1.0,
 0.05: 1.0,
 0.1: 1.0,
 0.2: 1.0,
 0.5: 1.0,
 0.8: 1.0,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


1048576/1048576 [==============================] - 681s 649us/step - loss: 0.0021 - accuracy: 0.9993
Steps 1048576: 0.999


{0: 0.094,
 0.01: 1.0,
 0.02: 1.0,
 0.05: 1.0,
 0.1: 1.0,
 0.2: 1.0,
 0.5: 1.0,
 0.8: 1.0,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


CPU times: user 15min 39s, sys: 59.7 s, total: 16min 38s
Wall time: 12min 13s


In [6]:
%%time
get_diff(256, 2**4)
get_diff(256, 2**8)
get_diff(256, 2**12)
get_diff(256, 2**16)
get_diff(256, 2**20)

16/16 [==============================] - 0s 889us/step - loss: 0.8804 - accuracy: 0.4629
Steps 16: 0.463


{0: 0.0,
 0.01: 0.0,
 0.02: 0.0,
 0.05: 0.0,
 0.1: 0.003,
 0.2: 0.046,
 0.5: 0.602,
 0.8: 0.944,
 0.9: 0.983,
 0.95: 0.996,
 0.99: 1.0,
 1: 1.0}


256/256 [==============================] - 0s 671us/step - loss: 0.6861 - accuracy: 0.5677
Steps 256: 0.568


{0: 0.0,
 0.01: 0.0,
 0.02: 0.0,
 0.05: 0.0,
 0.1: 0.008,
 0.2: 0.031,
 0.5: 0.561,
 0.8: 0.987,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


4096/4096 [==============================] - 3s 653us/step - loss: 0.6042 - accuracy: 0.6532
Steps 4096: 0.653


...


65536/65536 [==============================] - 42s 634us/step - loss: 0.2368 - accuracy: 0.8991
Steps 65536: 0.899


{0: 0.023,
 0.01: 0.725,
 0.02: 0.807,
 0.05: 0.906,
 0.1: 0.978,
 0.2: 0.997,
 0.5: 1.0,
 0.8: 1.0,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


1048576/1048576 [==============================] - 664s 633us/step - loss: 0.0168 - accuracy: 0.9932
Steps 1048576: 0.993


{0: 0.035,
 0.01: 1.0,
 0.02: 1.0,
 0.05: 1.0,
 0.1: 1.0,
 0.2: 1.0,
 0.5: 1.0,
 0.8: 1.0,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


CPU times: user 15min 15s, sys: 54.8 s, total: 16min 10s
Wall time: 11min 52s


In [7]:
%%time
get_diff(1024, 2**4)
get_diff(1024, 2**8)
get_diff(1024, 2**12)
get_diff(1024, 2**16)
get_diff(1024, 2**20)

16/16 [==============================] - 0s 923us/step - loss: 1.0524 - accuracy: 0.4668
Steps 16: 0.467


{0: 0.0,
 0.01: 0.01,
 0.02: 0.013,
 0.05: 0.03,
 0.1: 0.092,
 0.2: 0.159,
 0.5: 0.252,
 0.8: 0.834,
 0.9: 0.96,
 0.95: 0.978,
 0.99: 0.994,
 1: 1.0}


256/256 [==============================] - 0s 659us/step - loss: 0.7219 - accuracy: 0.5182
Steps 256: 0.518


{0: 0.0,
 0.01: 0.0,
 0.02: 0.0,
 0.05: 0.0,
 0.1: 0.0,
 0.2: 0.004,
 0.5: 0.537,
 0.8: 0.996,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


4096/4096 [==============================] - 3s 649us/step - loss: 0.6738 - accuracy: 0.5596
Steps 4096: 0.560


...


65536/65536 [==============================] - 43s 646us/step - loss: 0.5639 - accuracy: 0.6858
Steps 65536: 0.686


{0: 0.0,
 0.01: 0.078,
 0.02: 0.109,
 0.05: 0.167,
 0.1: 0.226,
 0.2: 0.332,
 0.5: 0.71,
 0.8: 0.98,
 0.9: 0.998,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


1048576/1048576 [==============================] - 664s 633us/step - loss: 0.2460 - accuracy: 0.9007
Steps 1048576: 0.901


{0: 0.009,
 0.01: 0.666,
 0.02: 0.715,
 0.05: 0.786,
 0.1: 0.84,
 0.2: 0.891,
 0.5: 0.94,
 0.8: 0.966,
 0.9: 0.973,
 0.95: 0.979,
 0.99: 0.988,
 1: 1.0}


CPU times: user 15min 15s, sys: 53.6 s, total: 16min 9s
Wall time: 11min 54s


In [8]:
%%time
get_diff(4096, 2**4)
get_diff(4096, 2**8)
get_diff(4096, 2**12)
get_diff(4096, 2**16)
get_diff(4096, 2**20)

16/16 [==============================] - 0s 825us/step - loss: 1.0011 - accuracy: 0.4863
Steps 16: 0.486


{0: 0.0,
 0.01: 0.002,
 0.02: 0.004,
 0.05: 0.008,
 0.1: 0.013,
 0.2: 0.039,
 0.5: 0.427,
 0.8: 0.923,
 0.9: 0.979,
 0.95: 0.987,
 0.99: 1.0,
 1: 1.0}


256/256 [==============================] - 0s 670us/step - loss: 0.7204 - accuracy: 0.5070
Steps 256: 0.507


{0: 0.0,
 0.01: 0.0,
 0.02: 0.0,
 0.05: 0.0,
 0.1: 0.0,
 0.2: 0.01,
 0.5: 0.788,
 0.8: 0.996,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


4096/4096 [==============================] - 3s 687us/step - loss: 0.6926 - accuracy: 0.5300
Steps 4096: 0.530


...


65536/65536 [==============================] - 43s 647us/step - loss: 0.6398 - accuracy: 0.6070
Steps 65536: 0.607


{0: 0.0,
 0.01: 0.002,
 0.02: 0.008,
 0.05: 0.043,
 0.1: 0.084,
 0.2: 0.139,
 0.5: 0.658,
 0.8: 0.971,
 0.9: 0.998,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


1048576/1048576 [==============================] - 668s 637us/step - loss: 0.5351 - accuracy: 0.6957
Steps 1048576: 0.696


{0: 0.0,
 0.01: 0.108,
 0.02: 0.132,
 0.05: 0.171,
 0.1: 0.222,
 0.2: 0.321,
 0.5: 0.732,
 0.8: 0.977,
 0.9: 0.995,
 0.95: 0.999,
 0.99: 1.0,
 1: 1.0}


CPU times: user 15min 24s, sys: 54.4 s, total: 16min 18s
Wall time: 12min 2s


In [9]:
%%time
get_diff(16384, 2**4)
get_diff(16384, 2**8)
get_diff(16384, 2**12)
get_diff(16384, 2**16)
get_diff(16384, 2**20)

16/16 [==============================] - 0s 837us/step - loss: 0.9974 - accuracy: 0.4766
Steps 16: 0.477


{0: 0.0,
 0.01: 0.0,
 0.02: 0.0,
 0.05: 0.0,
 0.1: 0.001,
 0.2: 0.02,
 0.5: 0.515,
 0.8: 0.935,
 0.9: 0.972,
 0.95: 0.994,
 0.99: 1.0,
 1: 1.0}


256/256 [==============================] - 0s 671us/step - loss: 0.7452 - accuracy: 0.5038
Steps 256: 0.504


{0: 0.0,
 0.01: 0.0,
 0.02: 0.0,
 0.05: 0.0,
 0.1: 0.001,
 0.2: 0.01,
 0.5: 0.671,
 0.8: 0.987,
 0.9: 0.995,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


4096/4096 [==============================] - 3s 667us/step - loss: 0.6822 - accuracy: 0.5724
Steps 4096: 0.572


...


65536/65536 [==============================] - 42s 640us/step - loss: 0.6432 - accuracy: 0.6118
Steps 65536: 0.612


{0: 0.0,
 0.01: 0.001,
 0.02: 0.013,
 0.05: 0.058,
 0.1: 0.104,
 0.2: 0.132,
 0.5: 0.73,
 0.8: 0.993,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}


1048576/1048576 [==============================] - 675s 644us/step - loss: 0.5657 - accuracy: 0.6568
Steps 1048576: 0.657


{0: 0.0,
 0.01: 0.091,
 0.02: 0.116,
 0.05: 0.141,
 0.1: 0.162,
 0.2: 0.205,
 0.5: 0.724,
 0.8: 0.994,
 0.9: 0.998,
 0.95: 0.999,
 0.99: 1.0,
 1: 1.0}


CPU times: user 15min 50s, sys: 54.2 s, total: 16min 45s
Wall time: 12min 26s


In [10]:
%%time
for num_nodes, num_layers in [(64,4)]:
    for train_size in [2**6, 2**8, 2**10, 2**12, 2**14, 2**16, 2**18]:
        for discrim_step in [2**4, 2**8, 2**12, 2**16]:
            print(num_nodes, num_layers, train_size, discrim_step)
            get_diff(train_size, discrim_step, num_nodes, num_layers)
            print()
        print('*'*50)
        print()

64 4 64 16
16/16 [==============================] - 0s 1ms/step - loss: 0.6957 - accuracy: 0.5156
Steps 16: 0.516


{0: 0.0,
 0.01: 0.0,
 0.02: 0.0,
 0.05: 0.0,
 0.1: 0.0,
 0.2: 0.0,
 0.5: 0.536,
 0.8: 1.0,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}



64 4 64 256
256/256 [==============================] - 1s 848us/step - loss: 0.6048 - accuracy: 0.6394
Steps 256: 0.639


{0: 0.0,
 0.01: 0.0,
 0.02: 0.005,
 0.05: 0.031,
 0.1: 0.057,
 0.2: 0.224,
 0.5: 0.568,
 0.8: 1.0,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}



64 4 64 4096
4096/4096 [==============================] - 4s 796us/step - loss: 0.2575 - accuracy: 0.8856
Steps 4096: 0.886


...



64 4 262144 4096
4096/4096 [==============================] - 4s 793us/step - loss: 0.0046 - accuracy: 0.9998
Steps 4096: 1.000


{0: 0.0,
 0.01: 1.0,
 0.02: 1.0,
 0.05: 1.0,
 0.1: 1.0,
 0.2: 1.0,
 0.5: 1.0,
 0.8: 1.0,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}



64 4 262144 65536
65536/65536 [==============================] - 50s 765us/step - loss: 0.1566 - accuracy: 0.9340
Steps 65536: 0.934


{0: 0.0,
 0.01: 0.999,
 0.02: 1.0,
 0.05: 1.0,
 0.1: 1.0,
 0.2: 1.0,
 0.5: 1.0,
 0.8: 1.0,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}



**************************************************

CPU times: user 15min 44s, sys: 54.8 s, total: 16min 39s
Wall time: 13min 15s


In [11]:
%%time
for num_nodes, num_layers in [(128,6)]:
    for train_size in [2**6, 2**8, 2**10, 2**12, 2**14, 2**16, 2**18]:
        for discrim_step in [2**4, 2**8, 2**12, 2**16]:
            print(num_nodes, num_layers, train_size, discrim_step)
            get_diff(train_size, discrim_step, num_nodes, num_layers)
            print()
        print('*'*50)
        print()

128 6 64 16
16/16 [==============================] - 0s 2ms/step - loss: 0.6893 - accuracy: 0.5742
Steps 16: 0.574


{0: 0.0,
 0.01: 0.0,
 0.02: 0.0,
 0.05: 0.0,
 0.1: 0.0,
 0.2: 0.0,
 0.5: 0.531,
 0.8: 1.0,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}



128 6 64 256
256/256 [==============================] - 1s 1ms/step - loss: 0.5731 - accuracy: 0.6864
Steps 256: 0.686


{0: 0.0,
 0.01: 0.0,
 0.02: 0.0,
 0.05: 0.016,
 0.1: 0.193,
 0.2: 0.286,
 0.5: 0.589,
 0.8: 0.969,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}



128 6 64 4096
4096/4096 [==============================] - 6s 1ms/step - loss: 0.1955 - accuracy: 0.9194
Steps 4096: 0.919


...



128 6 262144 4096
4096/4096 [==============================] - 6s 1ms/step - loss: 0.0053 - accuracy: 0.9995
Steps 4096: 0.999


{0: 0.0,
 0.01: 1.0,
 0.02: 1.0,
 0.05: 1.0,
 0.1: 1.0,
 0.2: 1.0,
 0.5: 1.0,
 0.8: 1.0,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}



128 6 262144 65536
65536/65536 [==============================] - 81s 1ms/step - loss: 0.1555 - accuracy: 0.9342
Steps 65536: 0.934


{0: 0.0,
 0.01: 1.0,
 0.02: 1.0,
 0.05: 1.0,
 0.1: 1.0,
 0.2: 1.0,
 0.5: 1.0,
 0.8: 1.0,
 0.9: 1.0,
 0.95: 1.0,
 0.99: 1.0,
 1: 1.0}



**************************************************

CPU times: user 22min 30s, sys: 1min 3s, total: 23min 34s
Wall time: 17min 36s
